In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape
from keras.layers.merge import concatenate, multiply,add
import tensorflow as tf
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


import rpy2
import rpy2.robjects.numpy2ri
from stldecompose import decompose
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
stats = importr('stats')
stl=stats.stl
ts =stats.ts

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data= pd.read_csv("Data/FromMohammed/cif_dataset_complete.csv",header=None)
predictions = pd.read_csv("Data/FromMohammed/theta_25_horg.csv",index_col=0,skiprows = [1])
print(data.shape)
print(predictions.shape)

(72, 123)
(72, 90)


# Case 2: Adding Expert Knowledge to layer 1

## 2.1: Using Concatenation 

In [5]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape1=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(horizon+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

##############################################################################################################################
        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l][0]=(train_pred1[l][0]+train_pred1[l][6])/2
                train_pred[l][1]=(train_pred1[l][1]+train_pred1[l][7])/2
                train_pred[l][2]=(train_pred1[l][2]+train_pred1[l][8])/2
                train_pred[l][3]=(train_pred1[l][3]+train_pred1[l][9])/2
                train_pred[l][4]=(train_pred1[l][4]+train_pred1[l][10])/2
                train_pred[l][5]=(train_pred1[l][5]+train_pred1[l][11])/2
            for l in range(val_pred1.shape[0]):
                val_pred[l][0]=(val_pred1[l][0]+val_pred1[l][6])/2
                val_pred[l][1]=(val_pred1[l][1]+val_pred1[l][7])/2
                val_pred[l][2]=(val_pred1[l][2]+val_pred1[l][8])/2
                val_pred[l][3]=(val_pred1[l][3]+val_pred1[l][9])/2
                val_pred[l][4]=(val_pred1[l][4]+val_pred1[l][10])/2
                val_pred[l][5]=(val_pred1[l][5]+val_pred1[l][11])/2
            for l in range(test_pred1.shape[0]):
                test_pred[l][0]=(test_pred1[l][0]+test_pred1[l][6])/2
                test_pred[l][1]=(test_pred1[l][1]+test_pred1[l][7])/2
                test_pred[l][2]=(test_pred1[l][2]+test_pred1[l][8])/2
                test_pred[l][3]=(test_pred1[l][3]+test_pred1[l][9])/2
                test_pred[l][4]=(test_pred1[l][4]+test_pred1[l][10])/2
                test_pred[l][5]=(test_pred1[l][5]+test_pred1[l][11])/2
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
        #train_pred=train_pred1
        #val_pred=val_pred1
        #test_pred=test_pred1
        
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)
        #train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        #val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        #test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF021.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF021.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape1[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction021.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape1)/72)    
#print(smape1)
#print(smape2)

2019-08-21 00:50:31.462819


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:02<02:53,  2.44s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:05<02:54,  2.50s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [02:00<06:11,  7.58s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [02:08<06:16,  7.84s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [02:17<06:22,  8.14s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [06:24<06:02, 13.94s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [06:40<05:59, 14.38s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [06:55<05:51, 14.65s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [13:09<01:00, 20.30s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [13:31<00:41, 20.57s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [13:53<00:21, 21.03s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_214 (Dense)               (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
dense_215 (Dense)               (None, 16)           144         dense_214[0][0]                  
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_216 

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [14:15<00:00, 21.33s/it]


2019-08-21 01:04:46.799319
-----------------
SMAPE1: (DeepEX approach)
13.870205177320965
SMAPE2 (NZR approach)
13.870205177320964
MSE1:
4016571544637.317
13.870205177320965


## 2.2: Using Add 

In [5]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape2=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(horizon+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

##############################################################################################################################
        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l][0]=(train_pred1[l][0]+train_pred1[l][6])/2
                train_pred[l][1]=(train_pred1[l][1]+train_pred1[l][7])/2
                train_pred[l][2]=(train_pred1[l][2]+train_pred1[l][8])/2
                train_pred[l][3]=(train_pred1[l][3]+train_pred1[l][9])/2
                train_pred[l][4]=(train_pred1[l][4]+train_pred1[l][10])/2
                train_pred[l][5]=(train_pred1[l][5]+train_pred1[l][11])/2
            for l in range(val_pred1.shape[0]):
                val_pred[l][0]=(val_pred1[l][0]+val_pred1[l][6])/2
                val_pred[l][1]=(val_pred1[l][1]+val_pred1[l][7])/2
                val_pred[l][2]=(val_pred1[l][2]+val_pred1[l][8])/2
                val_pred[l][3]=(val_pred1[l][3]+val_pred1[l][9])/2
                val_pred[l][4]=(val_pred1[l][4]+val_pred1[l][10])/2
                val_pred[l][5]=(val_pred1[l][5]+val_pred1[l][11])/2
            for l in range(test_pred1.shape[0]):
                test_pred[l][0]=(test_pred1[l][0]+test_pred1[l][6])/2
                test_pred[l][1]=(test_pred1[l][1]+test_pred1[l][7])/2
                test_pred[l][2]=(test_pred1[l][2]+test_pred1[l][8])/2
                test_pred[l][3]=(test_pred1[l][3]+test_pred1[l][9])/2
                test_pred[l][4]=(test_pred1[l][4]+test_pred1[l][10])/2
                test_pred[l][5]=(test_pred1[l][5]+test_pred1[l][11])/2
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
        #train_pred=train_pred1
        #val_pred=val_pred1
        #test_pred=test_pred1
        
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)
        #train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        #val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        #test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        #encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        #branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF022.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF022.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape2[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction022.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape2)/72)    
#print(smape1)
#print(smape2)

2019-08-21 01:08:07.185551


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:02<02:47,  2.36s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:04<02:47,  2.39s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [02:00<06:10,  7.55s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [02:08<06:17,  7.87s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [02:17<06:19,  8.08s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [06:25<06:02, 13.95s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [06:40<05:56, 14.27s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [06:56<05:50, 14.62s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [13:11<01:00, 20.30s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [13:33<00:41, 20.56s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [13:54<00:20, 20.86s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_214 (Dense)               (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_215 (Dense)               (None, 16)           144         dense_214[0][0]                  
__________________________________________________________________________________________________
conv1d_214

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [14:16<00:00, 21.12s/it]


2019-08-21 01:22:23.619815
-----------------
SMAPE1: (DeepEX approach)
14.120531693003743
SMAPE2 (NZR approach)
14.120531693003743
MSE1:
5137419093619.928
14.120531693003743


# Case 3 --> Using Concat

In [5]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape3=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(horizon+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

##############################################################################################################################
        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l][0]=(train_pred1[l][0]+train_pred1[l][6])/2
                train_pred[l][1]=(train_pred1[l][1]+train_pred1[l][7])/2
                train_pred[l][2]=(train_pred1[l][2]+train_pred1[l][8])/2
                train_pred[l][3]=(train_pred1[l][3]+train_pred1[l][9])/2
                train_pred[l][4]=(train_pred1[l][4]+train_pred1[l][10])/2
                train_pred[l][5]=(train_pred1[l][5]+train_pred1[l][11])/2
            for l in range(val_pred1.shape[0]):
                val_pred[l][0]=(val_pred1[l][0]+val_pred1[l][6])/2
                val_pred[l][1]=(val_pred1[l][1]+val_pred1[l][7])/2
                val_pred[l][2]=(val_pred1[l][2]+val_pred1[l][8])/2
                val_pred[l][3]=(val_pred1[l][3]+val_pred1[l][9])/2
                val_pred[l][4]=(val_pred1[l][4]+val_pred1[l][10])/2
                val_pred[l][5]=(val_pred1[l][5]+val_pred1[l][11])/2
            for l in range(test_pred1.shape[0]):
                test_pred[l][0]=(test_pred1[l][0]+test_pred1[l][6])/2
                test_pred[l][1]=(test_pred1[l][1]+test_pred1[l][7])/2
                test_pred[l][2]=(test_pred1[l][2]+test_pred1[l][8])/2
                test_pred[l][3]=(test_pred1[l][3]+test_pred1[l][9])/2
                test_pred[l][4]=(test_pred1[l][4]+test_pred1[l][10])/2
                test_pred[l][5]=(test_pred1[l][5]+test_pred1[l][11])/2
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
        #train_pred=train_pred1
        #val_pred=val_pred1
        #test_pred=test_pred1
        
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)
        #train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        #val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        #test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        #branch_0=concatenate([branch_0,encoded32],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_1=concatenate([branch_1,encoded32],axis=1)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF023.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF023.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape3[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction023.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape3)/72)    
#print(smape1)
#print(smape2)

2019-08-21 01:50:49.334805


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:02<02:52,  2.44s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:05<02:53,  2.48s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [02:01<06:12,  7.60s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [02:09<06:18,  7.88s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [02:18<06:26,  8.22s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [06:31<06:03, 13.96s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [06:46<05:59, 14.37s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [07:01<05:50, 14.60s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [13:21<01:03, 21.12s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [13:43<00:42, 21.26s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [14:05<00:21, 21.60s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_214 (Dense)               (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_215 (Dense)               (None, 16)           144         dense_214[0][0]                  
__________________________________________________________________________________________________
conv1d_214

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [14:27<00:00, 21.78s/it]


2019-08-21 02:05:17.317581
-----------------
SMAPE1: (DeepEX approach)
14.150533181116828
SMAPE2 (NZR approach)
14.150533181116828
MSE1:
8483554123009.014
14.150533181116828


# Case 4 --> Using Concat

In [8]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape4=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(horizon+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

##############################################################################################################################
        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l][0]=(train_pred1[l][0]+train_pred1[l][6])/2
                train_pred[l][1]=(train_pred1[l][1]+train_pred1[l][7])/2
                train_pred[l][2]=(train_pred1[l][2]+train_pred1[l][8])/2
                train_pred[l][3]=(train_pred1[l][3]+train_pred1[l][9])/2
                train_pred[l][4]=(train_pred1[l][4]+train_pred1[l][10])/2
                train_pred[l][5]=(train_pred1[l][5]+train_pred1[l][11])/2
            for l in range(val_pred1.shape[0]):
                val_pred[l][0]=(val_pred1[l][0]+val_pred1[l][6])/2
                val_pred[l][1]=(val_pred1[l][1]+val_pred1[l][7])/2
                val_pred[l][2]=(val_pred1[l][2]+val_pred1[l][8])/2
                val_pred[l][3]=(val_pred1[l][3]+val_pred1[l][9])/2
                val_pred[l][4]=(val_pred1[l][4]+val_pred1[l][10])/2
                val_pred[l][5]=(val_pred1[l][5]+val_pred1[l][11])/2
            for l in range(test_pred1.shape[0]):
                test_pred[l][0]=(test_pred1[l][0]+test_pred1[l][6])/2
                test_pred[l][1]=(test_pred1[l][1]+test_pred1[l][7])/2
                test_pred[l][2]=(test_pred1[l][2]+test_pred1[l][8])/2
                test_pred[l][3]=(test_pred1[l][3]+test_pred1[l][9])/2
                test_pred[l][4]=(test_pred1[l][4]+test_pred1[l][10])/2
                test_pred[l][5]=(test_pred1[l][5]+test_pred1[l][11])/2
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
        #train_pred=train_pred1
        #val_pred=val_pred1
        #test_pred=test_pred1
        
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)
        #train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        #val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        #test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        #branch_0=concatenate([branch_0,encoded32],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=concatenate([branch_1,encoded32],axis=1)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_2=concatenate([branch_2,encoded32],axis=1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF024.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF024.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape4[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction024.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape4)/72)    
#print(smape1)
#print(smape2)

2019-08-21 02:06:00.117211


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:23<28:11, 23.83s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:48<28:15, 24.22s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [10:52<25:48, 31.59s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [11:24<25:28, 31.85s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [11:55<24:41, 31.52s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [24:11<16:34, 38.26s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [24:51<16:08, 38.73s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [25:31<15:38, 39.11s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [40:42<02:19, 46.65s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [41:31<01:34, 47.14s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [42:19<00:47, 47.33s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_430 (Dense)               (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
conv1d_430 (Conv1D)             (None, 6, 32)        128         input_data[0][0]                 
__________________________________________________________________________________________________
dense_431 

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [43:08<00:00, 47.97s/it]


2019-08-21 02:49:08.603680
-----------------
SMAPE1: (DeepEX approach)
14.149791702673964
SMAPE2 (NZR approach)
14.149791702673962
MSE1:
4114043215819.5166
14.149791702673964


# Case 5 --> Using Concat

In [9]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape5=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(horizon+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

##############################################################################################################################
        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l][0]=(train_pred1[l][0]+train_pred1[l][6])/2
                train_pred[l][1]=(train_pred1[l][1]+train_pred1[l][7])/2
                train_pred[l][2]=(train_pred1[l][2]+train_pred1[l][8])/2
                train_pred[l][3]=(train_pred1[l][3]+train_pred1[l][9])/2
                train_pred[l][4]=(train_pred1[l][4]+train_pred1[l][10])/2
                train_pred[l][5]=(train_pred1[l][5]+train_pred1[l][11])/2
            for l in range(val_pred1.shape[0]):
                val_pred[l][0]=(val_pred1[l][0]+val_pred1[l][6])/2
                val_pred[l][1]=(val_pred1[l][1]+val_pred1[l][7])/2
                val_pred[l][2]=(val_pred1[l][2]+val_pred1[l][8])/2
                val_pred[l][3]=(val_pred1[l][3]+val_pred1[l][9])/2
                val_pred[l][4]=(val_pred1[l][4]+val_pred1[l][10])/2
                val_pred[l][5]=(val_pred1[l][5]+val_pred1[l][11])/2
            for l in range(test_pred1.shape[0]):
                test_pred[l][0]=(test_pred1[l][0]+test_pred1[l][6])/2
                test_pred[l][1]=(test_pred1[l][1]+test_pred1[l][7])/2
                test_pred[l][2]=(test_pred1[l][2]+test_pred1[l][8])/2
                test_pred[l][3]=(test_pred1[l][3]+test_pred1[l][9])/2
                test_pred[l][4]=(test_pred1[l][4]+test_pred1[l][10])/2
                test_pred[l][5]=(test_pred1[l][5]+test_pred1[l][11])/2
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
        #train_pred=train_pred1
        #val_pred=val_pred1
        #test_pred=test_pred1
        
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)
        #train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        #val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        #test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_1=concatenate([branch_1,encoded32],axis=1)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_2=concatenate([branch_2,encoded32],axis=1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF025.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF025.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape5[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction025.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape5)/72)    
#print(smape1)
#print(smape2)

2019-08-21 02:49:11.103652


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                               | 1/72 [00:50<1:00:12, 50.88s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [01:42<59:30, 51.01s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [21:33<49:26, 60.55s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [22:35<48:48, 61.02s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [23:38<48:05, 61.40s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [47:34<31:13, 72.07s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [48:49<30:22, 72.91s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [50:03<29:17, 73.21s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|████████████████████████████████████████████████████████████████████████████▋   | 69/72 [1:16:51<04:02, 80.85s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|█████████████████████████████████████████████████████████████████████████████▊  | 70/72 [1:18:13<02:42, 81.25s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|██████████████████████████████████████████████████████████████████████████████▉ | 71/72 [1:19:35<01:21, 81.52s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_646 (Dense)               (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
dense_647 (Dense)               (None, 16)           144         dense_646[0][0]                  
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_648 

100%|████████████████████████████████████████████████████████████████████████████████| 72/72 [1:20:59<00:00, 82.12s/it]


2019-08-21 04:10:10.608399
-----------------
SMAPE1: (DeepEX approach)
14.45918800664452
SMAPE2 (NZR approach)
14.45918800664452
MSE1:
8396103462851.639
14.45918800664452


# Case 2: Using 6 times Concatenation

In [5]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape222=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(horizon+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

##############################################################################################################################
        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l][0]=(train_pred1[l][0]+train_pred1[l][6])/2
                train_pred[l][1]=(train_pred1[l][1]+train_pred1[l][7])/2
                train_pred[l][2]=(train_pred1[l][2]+train_pred1[l][8])/2
                train_pred[l][3]=(train_pred1[l][3]+train_pred1[l][9])/2
                train_pred[l][4]=(train_pred1[l][4]+train_pred1[l][10])/2
                train_pred[l][5]=(train_pred1[l][5]+train_pred1[l][11])/2
            for l in range(val_pred1.shape[0]):
                val_pred[l][0]=(val_pred1[l][0]+val_pred1[l][6])/2
                val_pred[l][1]=(val_pred1[l][1]+val_pred1[l][7])/2
                val_pred[l][2]=(val_pred1[l][2]+val_pred1[l][8])/2
                val_pred[l][3]=(val_pred1[l][3]+val_pred1[l][9])/2
                val_pred[l][4]=(val_pred1[l][4]+val_pred1[l][10])/2
                val_pred[l][5]=(val_pred1[l][5]+val_pred1[l][11])/2
            for l in range(test_pred1.shape[0]):
                test_pred[l][0]=(test_pred1[l][0]+test_pred1[l][6])/2
                test_pred[l][1]=(test_pred1[l][1]+test_pred1[l][7])/2
                test_pred[l][2]=(test_pred1[l][2]+test_pred1[l][8])/2
                test_pred[l][3]=(test_pred1[l][3]+test_pred1[l][9])/2
                test_pred[l][4]=(test_pred1[l][4]+test_pred1[l][10])/2
                test_pred[l][5]=(test_pred1[l][5]+test_pred1[l][11])/2
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
        #train_pred=train_pred1
        #val_pred=val_pred1
        #test_pred=test_pred1
        
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)
        #train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        #val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        #test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        #branch_0=add([branch_0,encoded32])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==71:
            model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF0222.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF0222.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape222[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction0222.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape222)/72)    
#print(smape1)
#print(smape2)

2019-08-21 12:45:25.116724


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|█▏                                                                                 | 1/72 [00:02<03:00,  2.54s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  3%|██▎                                                                                | 2/72 [00:05<03:06,  2.66s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 32%|██████████████████████████▏                                                       | 23/72 [02:06<06:29,  7.95s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 33%|███████████████████████████▎                                                      | 24/72 [02:16<06:37,  8.27s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 35%|████████████████████████████▍                                                     | 25/72 [02:24<06:38,  8.47s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 64%|████████████████████████████████████████████████████▍                             | 46/72 [06:47<06:30, 15.00s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 65%|█████████████████████████████████████████████████████▌                            | 47/72 [07:04<06:24, 15.39s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 67%|██████████████████████████████████████████████████████▋                           | 48/72 [07:20<06:14, 15.60s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 69/72 [14:19<01:07, 22.34s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [14:42<00:44, 22.33s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [15:04<00:22, 22.48s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_214 (Dense)               (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
dense_215 (Dense)               (None, 16)           144         dense_214[0][0]                  
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_216 

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [15:28<00:00, 22.82s/it]


2019-08-21 13:00:53.630415
-----------------
SMAPE1: (DeepEX approach)
14.506553397805128
SMAPE2 (NZR approach)
14.506553397805128
MSE1:
4074342361507.1953
14.506553397805128


## Error : when the size of horizon is 12

In [18]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape2=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        if n_val < horizon: #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            n_val = horizon#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            
            
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        
        ##########################$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$##########################

        #train = series_data[:-n_val+3]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        test = series_d[-(horizon+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                        
        
        
        #train_p = series_pred[:-n_val+3]
        val_p = series_pred[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%        
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

##############################################################################################################################       
        train_pred=train_pred1
        val_pred=val_pred1
        test_pred=test_pred1
        
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        #train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        #val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        #test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])
##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,4),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,encoded32])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,encoded32])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        model.summary()
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF022.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF022.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler3.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape2[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction022.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print (sum(Final_smape2)/72)    
    

2019-08-21 00:06:38.526056


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '



ValueError: Error when checking input: expected input_7 to have shape (6,) but got array with shape (12,)